In [3]:
import os
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model

Using TensorFlow backend.


In [29]:
os.getcwd()

'C:\\Users\\91868\\Desktop\\Hari Data science\\Image dataset'

In [28]:
os.chdir('C:\\Users\\91868\\Desktop\\Hari Data science\\Image dataset')

In [4]:
## Loading the pre-defined VGG model to extract feature and removing last layer

model = VGG16()
model.layers.pop()
model = Model(inputs=model.inputs, outputs=model.layers[-1].output)

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [7]:
## Reading all 8K images (it icnludes 6K training & 1K validation & 1K testing images)
## Extracting all features from images and storing into a dictonary as { dict[image_name] = features }


from os import listdir

features=dict()
for name in listdir('C:\\Users\\91868\\Desktop\\Hari Data science\\Image dataset\\Flicker8k_Dataset'):
        filename = 'C:\\Users\\91868\\Desktop\\Hari Data science\\Image dataset\\Flicker8k_Dataset' + '//' + name
        image = load_img(filename, target_size=(224, 224))
        image = img_to_array(image)
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        image = preprocess_input(image)
        feature = model.predict(image, verbose=0)
        image_id = name.split('.')[0]
        features[image_id] = feature

In [320]:
features['1000268201_693b08cb0e']

array([[2.5076475, 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ]], dtype=float32)

In [9]:
## Here is the function to devide all ( Train & Validation & Test features)

def features_devide(path):
    imgs = open(path,'r')
    images = imgs.read()
    imgs.close()
    
    images = images.split('\n')
    images1 = []
    for i in images:
        images1.append(i.split('.')[0])
    
    features1 = dict()
    for i in images1:
        if(i in features.keys()):
            features1[i] = features[i]
    return features1

Dev_features = features_devide('C:\\Users\\91868\\Desktop\\Hari Data science\\Image captions\\Flickr_8k.devImages.txt')
test_features = features_devide('C:\\Users\\91868\\Desktop\\Hari Data science\\Image captions\\Flickr_8k.testImages.txt')
Train_features = features_devide('C:\\Users\\91868\\Desktop\\Hari Data science\\Image captions\\Flickr_8k.trainImages.txt')

In [10]:
print(len(Dev_features))
print(len(test_features))
print(len(Train_features))

1000
1000
5999


In [11]:
## Reading all descriptions file and storing into an object

txt_file = open('C:\\Users\\91868\\Desktop\\Hari Data science\\Image captions\\Flickr8k.lemma.token.txt', 'r')
text = txt_file.read()
txt_file.close()

In [12]:
## storing all descriptions into a dictonary. single image have multiple descriptions those are taken from different sources

descriptions = dict()
for line in text.split('\n'):
    tokens = line.split('\t')
    image_id, image_desc = tokens[0], tokens[1:]
    image_id = image_id.split('.')[0]
    if image_id not in descriptions:
        descriptions[image_id] = image_desc
    else:
        descriptions[image_id].append(str(image_desc))

In [322]:
descriptions['1305564994_00513f9a5b']

['man in street racer armor be examine the tire of another racer motorbike',
 'two racer drive white bike down road',
 'two motorist be ride along on their vehicle that be oddly design and color',
 'two person be in small race car drive by green hill',
 'two person in race uniform in street car']

In [13]:
## Data Cleansing activity on all descriptions like ( converting all into lower, removing 1 length words etc )

import string
table = str.maketrans('', '', string.punctuation)
for key, desc_list in descriptions.items():
    for i in range(len(desc_list)):
        desc = desc_list[i]
        desc = desc.split()
        desc = [word.lower() for word in desc]
        desc = [w.translate(table) for w in desc]
        desc = [word for word in desc if len(word)>1]
        desc = [word for word in desc if word.isalpha()]
        desc_list[i] =  ' '.join(desc)

In [14]:
## Here is the function to devide all descriptions based on image_id

def descriptions_devide(features):
    descs = dict()
    for key,desc in descriptions.items():
        if key in features.keys():
            if(key in descs):
                descs[key].append(str(desc))
            else:
                descs[key] = desc
    return descs

In [15]:
Train_descriptions = descriptions_devide(Train_features)
Dev_descriptions = descriptions_devide(Dev_features)
test_descriptions = descriptions_devide(test_features)

print(len(Train_descriptions))
print(len(Dev_descriptions))
print(len(test_descriptions))

5999
1000
1000


In [16]:
## Here we are reading all descriptions into one list to understand total no.of words 
## tokenizer.fit_on_texts - will help us to provide indexing for all words ( it will be helpful in language generaion) 

from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical

all_desc = list()
for key in descriptions.keys():
    for d in descriptions[key]:
        all_desc.append(d)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_desc)

vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

6687


In [323]:
tokenizer.word_index

{'in': 1,
 'be': 2,
 'on': 3,
 'the': 4,
 'dog': 5,
 'and': 6,
 'man': 7,
 'with': 8,
 'of': 9,
 'two': 10,
 'boy': 11,
 'girl': 12,
 'woman': 13,
 'person': 14,
 'white': 15,
 'black': 16,
 'run': 17,
 'play': 18,
 'wear': 19,
 'stand': 20,
 'to': 21,
 'at': 22,
 'jump': 23,
 'water': 24,
 'child': 25,
 'young': 26,
 'red': 27,
 'an': 28,
 'brown': 29,
 'his': 30,
 'sit': 31,
 'blue': 32,
 'through': 33,
 'walk': 34,
 'while': 35,
 'hold': 36,
 'shirt': 37,
 'ball': 38,
 'down': 39,
 'little': 40,
 'ride': 41,
 'grass': 42,
 'look': 43,
 'snow': 44,
 'over': 45,
 'front': 46,
 'three': 47,
 'small': 48,
 'field': 49,
 'large': 50,
 'up': 51,
 'by': 52,
 'green': 53,
 'group': 54,
 'one': 55,
 'yellow': 56,
 'her': 57,
 'into': 58,
 'air': 59,
 'beach': 60,
 'mouth': 61,
 'near': 62,
 'player': 63,
 'rock': 64,
 'dress': 65,
 'bike': 66,
 'street': 67,
 'another': 68,
 'for': 69,
 'its': 70,
 'from': 71,
 'as': 72,
 'climb': 73,
 'watch': 74,
 'outside': 75,
 'other': 76,
 'hat': 77,
 

In [17]:
## this function will help us to understand what are the maximum length of words in all descriptions

def max_length(descriptions):
    return max(len(d.split()) for d in all_desc)

max_length = max_length(descriptions)
max_length

32

In [18]:
## This function used to create inputs and output for tarining he model.
## inputs (X1 - freatures of photo, X2 - input words index numbers from tokenizer ) & output ( y - output word )

## ex: photo_name - desc [ dog walking on beach]
## inputs: X1=features[photo_name] , X2=[ word_indexes of [dog] ] & output: y=[ word_index[walking] ]
## inputs: X1=features[photo_name] , X2=[ word_indexes of [dog & walking] ] & output: y=[ word_index[on] ]
## inputs: X1=features[photo_name] , X2=[ word_indexes of [dog & walking & on] ] & output: y=[ word_index[beach] ]


import numpy as np
def create_seq(tokenizer,max_length,features,vocab_size,descriptions):
    X1, X2, y = list(), list(), list()
    for key, desc_list in descriptions.items():
        for desc in desc_list:
            seq = tokenizer.texts_to_sequences([desc])[0]
            for i in range(1, len(seq)):
                in_seq, out_seq = seq[:i], seq[i]
                in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                X1.append(features[key][0])
                X2.append(in_seq)
                y.append(out_seq)
    return np.array(X1),np.array(X2),np.array(y)


X1train, X2train, ytrain = create_seq(tokenizer,max_length,Train_features,vocab_size,Train_descriptions)
X1test, X2test, ytest = create_seq(tokenizer,max_length,Dev_features,vocab_size,Dev_descriptions)

In [19]:
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers.merge import add
from keras.callbacks import ModelCheckpoint

inputs1 = Input(shape=(4096,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)
# sequence model
inputs2 = Input(shape=(max_length,))
se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)
# decoder model
decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)
# tie it together [image, seq] [word]
model = Model(inputs=[inputs1, inputs2], outputs=outputs)
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam')
# summarize model
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 32)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 4096)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 32, 256)      1711872     input_3[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 4096)         0           input_2[0][0]                    
____________________________________________________________________________________________

In [31]:
filepath = 'model-ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
# fit model
model.fit([X1train, X2train], ytrain, epochs=2, verbose=1, callbacks=[checkpoint], validation_data=([X1test, X2test], ytest))

Train on 240851 samples, validate on 40680 samples
Epoch 1/2
240851/240851 [==============================] - 1763s 7ms/step - loss: 4.6437 - val_loss: 4.3242

Epoch 00001: val_loss improved from inf to 4.32419, saving model to model-ep001-loss4.644-val_loss4.324.h5
Epoch 2/2
240851/240851 [==============================] - 1732s 7ms/step - loss: 3.9900 - val_loss: 4.1462

Epoch 00002: val_loss improved from 4.32419 to 4.14624, saving model to model-ep002-loss3.990-val_loss4.146.h5


In [33]:
from keras.models import load_model

model = load_model('model-ep002-loss3.990-val_loss4.146.h5')

In [330]:
# X1,X2 are inputs used in training of the model only

## Here is the error i am getting if give single input feature and single seq

t1 = X1[0]
t2 = X2[0]

print(t1.shape)
print(t2.shape)

model.predict([t1,t2])

(4096,)
(32,)


ValueError: Error when checking input: expected input_2 to have shape (4096,) but got array with shape (1,)

In [332]:
## Here is the thing if i give list of input features and sequences, i am getting predictions

t1 = X1[0:5]
t2 = X2[0:5]

print(np.array(t1).shape)
print(np.array(t2).shape)

model.predict([t1,t2])

(5, 4096)
(5, 32)


array([[2.8348632e-11, 7.3000274e-06, 1.4644441e-04, ..., 5.4512877e-11,
        4.0112712e-11, 6.6072490e-12],
       [9.2933228e-10, 2.3872139e-02, 2.6460943e-01, ..., 1.3597146e-09,
        3.7291636e-09, 1.5694909e-09],
       [4.2562100e-11, 2.7341448e-02, 2.6034545e-02, ..., 9.2738012e-11,
        2.2583951e-12, 1.3537351e-11],
       [2.3922363e-13, 1.5173565e-01, 8.2067643e-05, ..., 2.6396187e-13,
        8.9695590e-16, 3.6722256e-14],
       [1.1709565e-10, 9.2951432e-03, 2.8177460e-03, ..., 1.8542291e-10,
        1.5721802e-12, 1.6891324e-10]], dtype=float32)

In [315]:
## So i am getting predictins, if i provide list of input features.
## i have decided to make dummy of same feature and same input sequence

## Here [ Photo ] is input features of photo & [ seqa ] dummy sequence( startseq )
## Here i processed last prediction only from list of predictions added into input sequnce predicted till maximum length of words occured in training


photo = list()
seqa = list()
in_seq = 'startseq'
sequence = tokenizer.texts_to_sequences([in_seq])[0]
sequence = pad_sequences([sequence], maxlen=max_length)
seqa.append(sequence[0])
seqa.append(sequence[0])
photo.append(test_features['3385593926_d3e9c21170'][0])
photo.append(test_features['3385593926_d3e9c21170'][0])

for i in range(max_length):
    yhat = model.predict([photo,seqa], verbose=0)
    a1 = np.argmax(yhat[len(yhat)-1])
    for word,index in tokenizer.word_index.items():
        if(index == a1):
            in_seq += ' ' + word
    sequence = tokenizer.texts_to_sequences([in_seq])[0]
    sequence = pad_sequences([sequence], maxlen=max_length)
    seqa.append(sequence[0])
    photo.append(test_features['3385593926_d3e9c21170'][0])
print(in_seq)

startseq dog be run through the snow with its mouth open in the snow with his mouth open in the snow in the snow with the top of the background in the snow


In [333]:
## Here is last predicted word in every iterations

photo = list()
seqa = list()
in_seq = 'startseq'
sequence = tokenizer.texts_to_sequences([in_seq])[0]
sequence = pad_sequences([sequence], maxlen=max_length)
seqa.append(sequence[0])
seqa.append(sequence[0])
photo.append(test_features['3385593926_d3e9c21170'][0])
photo.append(test_features['3385593926_d3e9c21170'][0])

for i in range(max_length):
    yhat = model.predict([photo,seqa], verbose=0)
    a1 = np.argmax(yhat[len(yhat)-1])
    word = word_for_id(a1, tokenizer)
    print(word)
    in_seq += ' ' + word
    sequence = tokenizer.texts_to_sequences([in_seq])[0]
    sequence = pad_sequences([sequence], maxlen=max_length)
    seqa.append(sequence[0])
    photo.append(test_features['3385593926_d3e9c21170'][0])

dog
be
run
through
the
snow
with
its
mouth
open
in
the
snow
with
his
mouth
open
in
the
snow
in
the
snow
with
the
top
of
the
background
in
the
snow


In [334]:
## Error : I am predicting 32 words, ie: maximum no.of words for a description used in training

## Questions:

## for some of images i dont need to generate 32 words, caption can come in 10-15 words
## If i get 'none' as a predicted word, so there i can stop iteration and i can show the description

## If i need to get 'none' as predicted word what should i do